In [1]:
import pickle
new_df = pickle.load(open('new.pkl','rb'))
movies = pickle.load(open('movie_dict.pkl','rb'))
similarity = pickle.load(open('similarity.pkl','rb'))
print(new_df)

      movie_id                                     title  \
0        19995                                    Avatar   
1          285  Pirates of the Caribbean: At World's End   
2       206647                                   Spectre   
3        49026                     The Dark Knight Rises   
4        49529                               John Carter   
...        ...                                       ...   
4804      9367                               El Mariachi   
4805     72766                                 Newlyweds   
4806    231617                 Signed, Sealed, Delivered   
4807    126186                          Shanghai Calling   
4808     25975                         My Date with Drew   

                                                   tags  
0     in the 22nd century, a parapleg marin is dispa...  
1     captain barbossa, long believ to be dead, ha c...  
2     a cryptic messag from bond’ past send him on a...  
3     follow the death of district attorney har

In [2]:
def recommend_10(movie):
    movie_list = []
    index = new_df[new_df['title'] == movie].index[0]   #index of searched movie in df
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])  #index and cos_sim tuple
    for i in distances[1:11]:
        movie = new_df.iloc[i[0]].title
        movie_list.append(movie)
    return movie_list

In [3]:
from flask import Flask,render_template,redirect,request,url_for

app = Flask(__name__)

@app.route('/')
def search():
    return render_template('search.html')

@app.route('/submit', methods=['GET','POST'])
def submit():
    movie=''
    if request.method == 'POST':
        movie = request.form['search']
        movie_list = recommend_10(movie)
    return render_template('results.html',movies=movie_list)

@app.route('/details')
def details():
    movie = request.args.get('movie')
    return render_template('movie.html',movie = movie)


if __name__ == '__main__':
    print("in main")
    app.run()



in main
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Feb/2024 18:34:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Feb/2024 18:34:35] "GET /favicon.ico HTTP/1.1" 404 -


in /


In [ ]:
p = recommend_10("Aliens")
print(p)

Login

In [1]:
from flask import Flask
from flask_mysqldb import MySQL
from flask import render_template, request, redirect, url_for, flash, session
import hashlib

app = Flask(__name__)

app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = '@gArwal80okD3p0rt'
app.config['MYSQL_DB'] = 'movie_rec'

app.config['MYSQL_CURSORCLASS'] = 'DictCursor'

mysql = MySQL(app)

app.secret_key = 'your_secret_key'

# Hashing function
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        # password = hashlib.sha256(request.form['password'].encode()).hexdigest()
        password = request.form['password']
        name = request.form['name']
        print(username,password,name)
        # name = request.args.get('name')
        # print(name)
        # username = request.form['username']
        # request.form['password']
        # username = request.args.get('username')
        # print(username)
        # password = hashlib.sha256(request.args.get('password').encode()).hexdigest()
        # password = hash_password(request.form.get('password'))
        # password = request.args.get('password')
        # print(password)
        # interested_genres = request.form.getlist('genre')  # Assuming your checkboxes have name='genre'
        # favorite_movie = request.form['favorite_movie']

        # Convert the list of genres to a comma-separated string
        # interested_genres_str = ', '.join(interested_genres)

        try:
            cur = mysql.connection.cursor()
            cur.execute("INSERT INTO users (uname, username, passwords) VALUES (%s, %s, %s)",
                        (name, username, password))
            mysql.connection.commit()
            cur.close()

            flash('Registration successful. You can now login.', 'success')
            return redirect(url_for('login'))
        except Exception as e:
            flash('Registration failed. Please try again later.', 'danger')
            print(f"An error occurred during registration: {e}")
            
    return render_template('register.html')


@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        # password = hashlib.sha256(request.form['password'].encode()).hexdigest()
        password = request.form['password']
        print(username)
        print(password)

        try:
            cur = mysql.connection.cursor()
            cur.execute("SELECT * FROM users WHERE username = %s AND passwords = %s", (username, password))
            user = cur.fetchone()
            cur.close()

            if user:
                session['user_id'] = user['username']
                flash('Login successful', 'success')
                return redirect(url_for('dashboard'))
            else:
                flash('Invalid credentials', 'danger')
        except Exception as e:
            flash('Login failed. Please try again later.', 'danger')
            print(f"An error occurred during login: {e}")
    return render_template('login.html')


@app.route('/logout')
def logout():
    session.clear()
    flash('You have been logged out.', 'info')
    return redirect(url_for('login'))

@app.route('/')
def index():
    return redirect(url_for('login'))

@app.route('/dashboard')
def dashboard():
    return render_template('search.html')

@app.route('/submit', methods=['GET','POST'])
def submit():
    movie=''
    if request.method == 'POST':
        movie = request.form['search']
        movie_list = recommend_10(movie)
    return render_template('results.html',movies=movie_list)

@app.route('/details')
def details():
    movie = request.args.get('movie')
    return render_template('movie.html',movie = movie)


@app.before_request
def require_login():
    allowed_routes = ['login', 'register','logout']  # add other allowed routes
    if 'user_id' not in session and request.endpoint not in allowed_routes:
        return redirect(url_for('login'))
    
if __name__ == "__main__":
    app.run()
    
    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Mar/2024 17:28:05] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [08/Mar/2024 17:28:05] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 17:28:05] "GET /favicon.ico HTTP/1.1" 302 -
127.0.0.1 - - [08/Mar/2024 17:28:05] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 17:28:10] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [08/Mar/2024 17:28:10] "GET /dashboard HTTP/1.1" 200 -


vijit
ninu


127.0.0.1 - - [08/Mar/2024 17:28:10] "GET /favicon.ico HTTP/1.1" 404 -
[2024-03-08 17:28:16,426] ERROR in app: Exception on /submit [POST]
Traceback (most recent call last):
  File "e:\Anaconda\envs\GG_2527\lib\site-packages\flask\app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
  File "e:\Anaconda\envs\GG_2527\lib\site-packages\flask\app.py", line 872, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "e:\Anaconda\envs\GG_2527\lib\site-packages\flask\app.py", line 870, in full_dispatch_request
    rv = self.dispatch_request()
  File "e:\Anaconda\envs\GG_2527\lib\site-packages\flask\app.py", line 855, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "C:\Users\hp\AppData\Local\Temp\ipykernel_17528\3639890253.py", line 109, in submit
    movie_list = recommend_10(movie)
NameError: name 'recommend_10' is not defined
127.0.0.1 - - [08/Mar/2024 17:28:16] "POS

In [4]:
import json
import urllib.request as urllib_request
import ssl
import pickle

def fetch_poster_path(api_key, base_url, movie_id):
    poster_url = f"{base_url}{movie_id}?api_key={api_key}&language=en-US"
    response = urllib_request.urlopen(poster_url)
    json_data = json.loads(response.read())

    # Extract the poster path from the API response
    poster_path = json_data.get('poster_path', '')
    
    return f"https://image.tmdb.org/t/p/w500/{poster_path}"

In [3]:
def recommend_10(movie):
    movie_list = []

    # Check if movie is None or an empty string
    if movie is None or not movie.strip():
        print("Error: Movie is None or empty.")
        return movie_list

    # Normalize movie title to lowercase and remove leading/trailing spaces
    movie = movie.strip().lower()

    # Check if movie exists in DataFrame
    if movie in new_df['title'].str.lower().values:
        index = new_df[new_df['title'].str.lower() == movie].index[0]
    else:
        # Movie not found in DataFrame
        print(f"Movie '{movie}' not found in DataFrame.")
        return movie_list

    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])

    for i in distances[1:11]:
        movie_index = i[0]
        movie_title = new_df.iloc[movie_index]['title']

        # Fetch the poster path dynamically from the TMDB API
        api_key = "ff1e18b29203a4007ed0df1907cccf71"
        base_url = "https://api.themoviedb.org/3/movie/"
        movie_id = new_df.iloc[movie_index]['movie_id']

        poster_path = fetch_poster_path(api_key, base_url, movie_id)

        movie_list.append({
            'title': movie_title,
            'poster_path': poster_path
        })

    return movie_list

In [5]:
from flask import Flask
from flask_mysqldb import MySQL
from flask import render_template, redirect, url_for, flash, session,request as flask_request
import hashlib

app = Flask(__name__)

app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = '@gArwal80okD3p0rt'
app.config['MYSQL_DB'] = 'movie_rec'

app.config['MYSQL_CURSORCLASS'] = 'DictCursor'

api_key = "ff1e18b29203a4007ed0df1907cccf71"
base_url = "https://api.themoviedb.org/3/discover/movie?api_key=" + api_key

mysql = MySQL(app)

app.secret_key = 'your_secret_key'

# Hashing function
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        name = request.form['name']
        try:
            cur = mysql.connection.cursor()
            cur.execute("INSERT INTO users (uname, username, passwords) VALUES (%s, %s, %s)",
                        (name, username, password))
            mysql.connection.commit()
            cur.close()

            flash('Registration successful. You can now login.', 'success')
            return redirect(url_for('login'))
        except Exception as e:
            flash('Registration failed. Please try again later.', 'danger')
            print(f"An error occurred during registration: {e}")
            
    return render_template('register.html')


@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        # password = hashlib.sha256(request.form['password'].encode()).hexdigest()
        password = request.form['password']
        print(username)
        print(password)

        try:
            cur = mysql.connection.cursor()
            cur.execute("SELECT * FROM users WHERE username = %s AND passwords = %s", (username, password))
            user = cur.fetchone()
            cur.close()

            if user:
                session['user_id'] = user['username']
                flash('Login successful', 'success')
                return redirect(url_for('dashboard'))
            else:
                flash('Invalid credentials', 'danger')
        except Exception as e:
            flash('Login failed. Please try again later.', 'danger')
            print(f"An error occurred during login: {e}")
    return render_template('login.html')


@app.route('/logout')
def logout():
    session.clear()
    flash('You have been logged out.', 'info')
    return redirect(url_for('login'))

@app.route('/')
def index():
    return redirect(url_for('login'))

@app.route('/dashboard', methods=['GET', 'POST'])
def dashboard():
    
    if flask_request.method == 'POST':
        print("in dashboard")
        search_query = flask_request.form.get('search')
        print(f"Search Query: {search_query}")

        recommended_movies = recommend_10(search_query)
        print(f"Recommended Movies: {recommended_movies}")

        return render_template("index.html", data=recommended_movies, search_query=search_query)

    ssl._create_default_https_context = ssl._create_unverified_context
    conn = urllib_request.urlopen(base_url)
    json_data = json.loads(conn.read())
    return render_template("index.html", data=json_data["results"], search_query=None)

@app.route('/submit', methods=['GET','POST'])
def submit():
    movie=''
    if request.method == 'POST':
        movie = request.form['search']
        movie_list = recommend_10(movie)
    return render_template('results.html',movies=movie_list)

@app.route('/details')
def details():
    movie = request.args.get('movie')
    return render_template('movie.html',movie = movie)


@app.before_request
def require_login():
    allowed_routes = ['login', 'register','logout']  # add other allowed routes
    if 'user_id' not in session and request.endpoint not in allowed_routes:
        return redirect(url_for('login'))
    
if __name__ == "__main__":
    app.run()
    
    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Mar/2024 17:29:31] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [08/Mar/2024 17:29:31] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 17:29:36] "POST /login HTTP/1.1" 302 -


vijit
ninu


127.0.0.1 - - [08/Mar/2024 17:29:36] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 17:29:36] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 17:29:48] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 17:30:02] "POST /register HTTP/1.1" 302 -
127.0.0.1 - - [08/Mar/2024 17:30:02] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 17:30:10] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [08/Mar/2024 17:30:10] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 17:30:10] "GET /static/style.css HTTP/1.1" 304 -


hello
vijit
